# Data Collection and Preprocessing

In this notebook, we use the Green Metric Tool API to collect and preprocess the power and traffic data captured while surfing on major website with and without an ad-blocker.


### Importing libraries

In [ ]:
import pandas as pd # For data manipulation
import numpy as np # For data manipulation
import requests # For HTTP Requests to the API
import seaborn as sns # For charts
from IPython.display import display # To dataframe displaying
import scipy.stats as stats # For stats
import pingouin as pg # For stats
from itertools import combinations
import matplotlib.pyplot as plt
from datetime import datetime, timezone
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

#Set display options to show all rows and columns
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)  # Auto-detect the width
pd.set_option('display.max_colwidth', None)  # Show full content of each column

- Define the URL and other constants to query the GMT API

In [58]:
URL = 'http://api.green-coding.internal:9142'
REPO = '/home/jason/memoire/simulator'
RUNS = '/v1/runs'
NOTES = '/v1/notes/'
MEASUREMENTS = '/v1/measurements/single/'

- Define functions to:
    - get all the runs infos (test_name, run_ids, time)
    - get all run ids of tests which names include a substring
    - get the test name of a given run id
    - get the notes from a given run id
    - print the logs of a given run id
    - get the measurment of a given run id

In [59]:
# Function to get all runs
def get_runs():
    try:
        # Fetch data from the API
        response = requests.get(URL + RUNS)
        response.raise_for_status()  # Check if the request was successful
        
        data = response.json()
        
        # Convert the relevant part of the data into a DataFrame
        runs = pd.DataFrame.from_dict(data['data'])
        runs = runs[[0, 1, 4]]
        runs.columns = ['id', 'name', 'time']
        # Convert the 'time' column (containing UTC timestamp strings) to UTC timestamps
        runs['time'] = pd.to_datetime(runs['time'])
        
        return runs
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
    except ValueError as e:
        print(f"Failed to decode JSON: {e}")

# Function to get all the run IDs for a test name containing a given substring
def get_ids(substring="", time=False):
    runs = get_runs()
    filtered_runs = runs[runs['name'].str.contains(substring, na=False)]
    if time:
        return filtered_runs['id'].tolist(), filtered_runs['time'].tolist()
    return filtered_runs['id'].tolist()

# Function to get the test name of a run given its ID
def get_name(id):
    runs = get_runs()
    run = runs[runs['id'] == id]
    
    if not run.empty:
        return run.iloc[0]['name']
    else:
        return None

# Function to create the notes (i.e., logs) DataFrame from an ID
def get_notes(id):
    try:
        response = requests.get(URL + NOTES + id)
        response.raise_for_status()  # Check if the request was successful
        data = response.json()
        notes = pd.DataFrame.from_dict(data['data'])
        
        # Set the appropriate column names
        notes.columns = ['id', 'container', 'log', 'time']

        # Sort the logs by time
        notes.sort_values(by='time', ascending=False, inplace=True)
        
        return notes
        
    except Exception as e:
        print(f"Failed to fetch or process notes for ID {id}: {e}")
        # Return an empty DataFrame with the expected column names
        return pd.DataFrame(columns=['id', 'container', 'log', 'time'])

# Function to print the logs for a given ID
def print_logs(id):
    # print the notes (logs) for the given ID
    notes_df = get_notes(id)
    
    if notes_df.empty:
        print(f"No logs found for ID {id}.")
        return
    
    # Print the logs in order
    for index, row in notes_df.iterrows():
        print(f"Time: {row['time']}, Log: {row['log']}")

# Function to get the measurement DataFrame from an ID
def get_measurements(id):
    try:
        response = requests.get(URL + MEASUREMENTS + id)
        response.raise_for_status()  # Check if the request was successful
        data = response.json()
        measurement = pd.DataFrame.from_dict(data['data'])
        
        # Set the appropriate column names
        measurement.columns = ['detail_name', 'time', 'metric', 'value', 'unit']
        return measurement

    except Exception as e:
        print(f"Failed to fetch measurements for ID {id}: {e}")
        # Return an empty DataFrame in case of an error
        return pd.DataFrame(columns=['detail_name', 'time', 'metric', 'value', 'unit'])

- Define a DataFrame to Map conditions (i.e. use of adblock) and Functional Units (Surfing to a given website) to Corresponding Tests and Actions to Monitor:

    EXAMPLE:
    - Considering the 'surf to google' functional unit with adblock enabled, we look up the entry `['enabled', 'google']`. This entry directs us to the test name `'topnews_adblock'` and specifies the name of the action to monitor: `'surf_to_https://google.com'`.

    - Considering the 'surf to yahoo' functional unit with adblock disabled, we look up the entry `['disabled', 'yahoo']`. This entry directs us to the test name `'topnews_noadblock'` and specifies the name of the action to monitor: `'surf_to_https://yahoo.com'`.


In [ ]:
ADBLOCK_CONDITION = ['enabled', 'disabled']
FUNCTIONAL_UNITS =     [
                        'google',
                        'yahoo',  
                        'bing',
                        'wikipedia',
                        'weather',
                        'cnn',
                        'foxnews',
                        'nytimes',
                        'sohu',
                        'taobao']

def get_test_name_and_action(adblock, functional_unit):
    test_name = "topnews_"
    action = None
    test_name += 'adblock' if adblock == 'enabled' else 'noadblock'

    if functional_unit == 'session':
        action = 'session'
    elif functional_unit == 'google':
        action = 'surf_to_' + 'https://google.com'
    elif functional_unit == 'yahoo':
        action = 'surf_to_' + 'https://yahoo.com'
    elif functional_unit == 'bing':
        action = 'surf_to_' + 'https://bing.com'
    elif functional_unit == 'wikipedia':
        action = 'surf_to_' + 'https://wikipedia.org'
    elif functional_unit == 'weather':
        action = 'surf_to_' + 'https://weather.com'
    elif functional_unit == 'cnn':
        action = 'surf_to_' + 'https://cnn.com'
    elif functional_unit == 'foxnews':
        action = 'surf_to_' + 'https://foxnews.com'
    elif functional_unit == 'nytimes':
        action = 'surf_to_' + 'https://nytimes.com'
    elif functional_unit == 'sohu':
        action = 'surf_to_' + 'https://sohu.com'
    elif functional_unit == 'taobao':
        action = 'surf_to_' + 'https://taobao.com'


    return test_name, action
    

# Generate all combinations of provider, adblock, pgp, and functional_unit
tests_and_actions = [
    [adblock, functional_unit] + list(get_test_name_and_action(adblock, functional_unit))
    for adblock in ADBLOCK_CONDITION
    for functional_unit in FUNCTIONAL_UNITS
]

# Create the DataFrame
TESTS_AND_ACTIONS = (pd.DataFrame(tests_and_actions, columns=['ADBLOCK_CONDITION', 'FUNCTIONAL_UNIT', 'TEST', 'ACTION']) \
    .set_index(['ADBLOCK_CONDITION', 'FUNCTIONAL_UNIT']))


TESTS_AND_ACTIONS

TEST  \
ADBLOCK_CONDITION FUNCTIONAL_UNIT                      
enabled           google             topnews_adblock   
                  yahoo              topnews_adblock   
                  bing               topnews_adblock   
                  wikipedia          topnews_adblock   
                  weather            topnews_adblock   
                  cnn                topnews_adblock   
                  foxnews            topnews_adblock   
                  nytimes            topnews_adblock   
                  sohu               topnews_adblock   
                  taobao             topnews_adblock   
disabled          google           topnews_noadblock   
                  yahoo            topnews_noadblock   
                  bing             topnews_noadblock   
                  wikipedia        topnews_noadblock   
                  weather          topnews_noadblock   
                  cnn              topnews_noadblock   
                  foxnews          topnews_noadblock   
                  nytimes          topnews_noadblock   
                  sohu             topnews_noadblock   
                  taobao           topnews_noadblock   

                                                          ACTION  
ADBLOCK_CONDITION FUNCTIONAL_UNIT                                 
enabled           google              surf_to_https://google.com  
                  yahoo                surf_to_https://yahoo.com  
                  bing                  surf_to_https://bing.com  
                  wikipedia        surf_to_https://wikipedia.org  
                  weather            surf_to_https://weather.com  
                  cnn                    surf_to_https://cnn.com  
                  foxnews            surf_to_https://foxnews.com  
                  nytimes            surf_to_https://nytimes.com  
                  sohu                  surf_to_https://sohu.com  
                  taobao              surf_to_https://taobao.com  
disabled          google              surf_to_https://google.com  
                  yahoo                surf_to_https://yahoo.com  
                  bing                  surf_to_https://bing.com  
                  wikipedia        surf_to_https://wikipedia.org  
                  weather            surf_to_https://weather.com  
                  cnn                    surf_to_https://cnn.com  
                  foxnews            surf_to_https://foxnews.com  
                  nytimes            surf_to_https://nytimes.com  
                  sohu                  surf_to_https://sohu.com  
                  taobao              surf_to_https://taobao.com

- Metadata Collection and Sample Counting :

    We aim to collect metadata related to energy measurements for different **functional units (websites)** with/without **ad-blocker**. The process includes:

    - **`collect_metadata`**: Gathers metadata for a functional unit under a given condition, including start and end timestamps, ensuring no errors occur during the functional unit execution to validate the sample.
    - **`collect_all_metadata`**: collect metadata all tests.

    Additionally, we monitor the number of valid samples collected:

    - **`count_distinct_samples`**: Counts valid and missing samples based on unique start and end timestamps.


In [61]:

def collect_metadata(adblock, functional_unit, test_name, action, from_date=None, till_date=None):
    """
    Collect metadata required to fetch energy measurements.
    This includes gathering the run ID and the time period when a functional unit was performed.
    A period is considered valid if no error occurred during the functional unit execution.

    Args:
        adblock (str): The adblock condition: enabled/disabled.
        functional_unit (str): The functional unit to monitor.
        test_name (str): The name of the test which perform the functional units for the provider under given adblock/pgp conditions.
        action (str): The action that triggers the start/end logs.
        from_date (str, optional): Start date in 'dd-mm-yyyy' format. If None, no start limit is applied.
        till_date (str, optional): End date in 'dd-mm-yyyy' format. If None, no end limit is applied.

    Returns:
        pd.DataFrame: A DataFrame containing metadata with columns:
                      ['TEST', 'RUN_ID', 'ADBLOCK_CONDITION',
                       'FUNCTIONAL_UNIT', 'START_TS', 'END_TS'].
    """
    
    columns = ['TEST', 'RUN_ID','ADBLOCK_CONDITION', 'FUNCTIONAL_UNIT', 'START_TS', 'END_TS']
    metadata_records = []  # List to store metadata records
    ids, timestamps = get_ids(test_name, time=True)  # Fetch the test run IDs and their corresponding timestamps

    for run_id, run_timestamp in zip(ids, timestamps, strict=True):

        # Check if the timestamp fits within the date range (if provided)
        if from_date and run_timestamp < pd.to_datetime(from_date, format="%d-%m-%Y", utc=True):
            continue  # Skip records that are before the 'from' time
        if till_date and run_timestamp > pd.to_datetime(till_date, format="%d-%m-%Y", utc=True):
            continue  # Skip records that are after the 'till' time

        logs = get_notes(run_id)  # Get logs for the run
        end_log_time = None  # Variable to track the latest "END" log timestamp

        for _, row in logs.iterrows():
            log_message = row['log']
            timestamp = row['time']

            # Detect end log for the specific action
            if f"END: {action}" in log_message:
                end_log_time = timestamp
                continue

            # Invalidate end time if there's an error during action execution
            if 'ERROR' in log_message or 'stacktrace' in log_message:
                end_log_time = None
                continue

            # Collect metadata once a valid start-end pair is found
            if f"START: {action}" in log_message and end_log_time is not None:
                metadata_records.append((test_name, run_id, adblock, functional_unit, timestamp, end_log_time))
                end_log_time = None  # Reset after recording
    
    return pd.DataFrame(metadata_records, columns=columns)



def collect_all_metadata(from_date=None, till_date=None):
    """
    Collect metadata for all (functional units, adblock_condition) combination.
    """
    metadata_list = [
        collect_metadata(adblock, functional_unit, row['TEST'], row['ACTION'], from_date, till_date)
        for (adblock, functional_unit), row in TESTS_AND_ACTIONS.iterrows()
    ]

    return pd.concat(metadata_list, ignore_index=True,)

def count_distinct_samples(metadata: pd.DataFrame):
    """
    Count the number of valid samples for each (functional_unit, adblock_condition) combination.
    A valid sample is defined as a unique pair of start and end timestamps for each (TEST, ACTION).

    Args:
        metadata (pd.DataFrame): The metadata DataFrame to count samples from.

    Returns:
        pd.DataFrame: A DataFrame containing the sample counts and missing sample counts.
    """
    return (metadata.groupby(['ADBLOCK_CONDITION', 'FUNCTIONAL_UNIT'])
                .agg(SAMPLE_COUNT=('START_TS', 'count'), 
                    MISSING_SAMPLES=('START_TS', lambda x: max(0, 100 - x.count())))  # Count missing samples
            )

In [ ]:
# Collect the full metadata DataFrame
# Tests performed in Switzerland on a ethernet link connection : '25-02-2025 - 27-02-2025'

metadata = collect_all_metadata(from_date='25-02-2025')

- First we check which metrics we have at our disposal, their name and units

In [8]:
# First we check which metric we have at our disposal
test_id = get_ids("topnews")[0]
print(get_measurements(test_id)[['metric', 'unit']].drop_duplicates())

Request failed: HTTPConnectionPool(host='api.green-coding.internal', port=9142): Max retries exceeded with url: /v1/runs (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x136c4ed50>: Failed to establish a new connection: [Errno 61] Connection refused'))


TypeError: 'NoneType' object is not subscriptable

- Then we rename those metrics and define our own computed metrics

In [18]:
METRICS = [
    'CPU_ENERGY',                   # cpu_energy_rapl_msr_component
    'MEMORY_ENERGY',                # memory_energy_rapl_msr_component
    'NETWORK_IO',                   # network_io_cgroup_container
    'ESTIMATED_MACHINE_ENERGY',     # psu_energy_ac_xgboost_machine
    'DURATION']

COMPUTED_METRICS = ['CPU_POWER',
                    'MEMORY_POWER',
                    'ESTIMATED_MACHINE_POWER']
UNITS = {
    'CPU_ENERGY': 'mJ',
    'MEMORY_ENERGY': 'mJ',
    'NETWORK_IO' : 'Byte',
    'ESTIMATED_MACHINE_ENERGY' : 'mJ',
    'DURATION' : 'µs',
    'CPU_POWER' : 'W',
    'MEMORY_POWER' : 'W',
    'ESTIMATED_MACHINE_POWER' : 'W'
}

- Collect and Organise Metric Data into a DataFrame :

    **Gathering Measurement Data**

    In the `get_data` function, we collect measurement data for each sample in the `metadata` DataFrame. We extract key metadata, including test names, run IDs, adblock conditions, functional units, and the start and end timestamps for each sample. For each time period, we then compute the energy- or data-related metrics—such as memory energy, CPU energy, estimated machine energy, and network I/O—using the `get_metric_stat` function.

    **Storing the Collected Data**

    We store the computed values in a DataFrame. The columns of this DataFrame are: `TEST`, `RUN_ID`, `FUNCTIONAL_UNIT`, `ADBLOCK_CONDITION`, `START_TS`, `DURATION`, `MEMORY_ENERGY`, `CPU_ENERGY`, `ESTIMATED_MACHINE_ENERGY`, and `NETWORK_IO`. Then we export it as a csv file


In [9]:
def get_metric_stat(measurements, metric, start_ts, end_ts, func):

    # Filter to keep records of the desired metric
    measurements = measurements[measurements['metric'] == metric]
    # Filter to keep records in desired period
    measurements = measurements[(measurements['time'] >= start_ts) & (measurements['time'] < end_ts)]
    # Compute the statistic
    stat = func(measurements['value'])

    return stat

def get_data(metadata:pd.DataFrame):

    data_records = []  # Store tuples for DataFrame creation

    for run_id, run_metadata in metadata.groupby('RUN_ID') :

        # Get measurment for the run
        run_measurements = get_measurements(run_id)
        run_measurements.sort_values(by='time', ascending=True, inplace=True)

        for _, row in run_metadata.iterrows():

            # Extract Metadata:
            test = row['TEST']
            adblock_condition = row['ADBLOCK_CONDITION'] 
            functional_unit = row['FUNCTIONAL_UNIT']
            start_ts = row['START_TS']
            end_ts = row['END_TS']
            duration = end_ts - start_ts

            # Compute Data
            memory_energy = get_metric_stat(run_measurements, 'memory_energy_rapl_msr_component', start_ts, end_ts, sum)
            cpu_energy = get_metric_stat(run_measurements, 'cpu_energy_rapl_msr_component', start_ts, end_ts, sum)
            estimated_machine_energy = get_metric_stat(run_measurements, 'psu_energy_ac_xgboost_machine', start_ts, end_ts, sum)
            network_io = get_metric_stat(run_measurements, 'network_io_cgroup_container', start_ts, end_ts, lambda x: x.iloc[-1] - x.iloc[0] if len(x) > 1 else 0)

            # Store records
            data_records.append((test,
                                run_id,
                                adblock_condition,
                                functional_unit,
                                start_ts,
                                duration,
                                memory_energy,
                                cpu_energy,
                                estimated_machine_energy,
                                network_io))
        
    data = pd.DataFrame(  columns = ['TEST',
                                    'RUN_ID',
                                    'ADBLOCK_CONDITION',
                                    'FUNCTIONAL_UNIT',
                                    'START_TS',
                                    'DURATION',
                                    'MEMORY_ENERGY',
                                    'CPU_ENERGY',
                                    'ESTIMATED_MACHINE_ENERGY',
                                    'NETWORK_IO'],
                        data = data_records).sort_values(by=['ADBLOCK_CONDITION',
                                                                'FUNCTIONAL_UNIT'])
    return data

In [66]:
data = get_data(metadata=metadata)
data.to_csv('2025_topnews_data.csv', index=False)